# Step 1: Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split

# Step 2: Load the Datasets

In [2]:
# 2.1 Load movie metadata with descriptions
# This CSV should include: movieId, title, genres, year, description.
movies_df = pd.read_csv('movielens_movies_with_descriptions.csv')
print("Movies with descriptions shape:", movies_df.shape)
print(movies_df.head())

Movies with descriptions shape: (3883, 5)
   movieId                        title                        genres  year  \
0        1                    Toy Story   Animation|Children's|Comedy  1995   
1        2                      Jumanji  Adventure|Children's|Fantasy  1995   
2        3             Grumpier Old Men                Comedy|Romance  1995   
3        4            Waiting to Exhale                  Comedy|Drama  1995   
4        5  Father of the Bride Part II                        Comedy  1995   

                                         description  
0  Led by Woody, Andy's toys live happily in his ...  
1  When siblings Judy and Peter discover an encha...  
2  A family wedding reignites the ancient feud be...  
3  Cheated on, mistreated and stepped on, the wom...  
4  Just when George Banks has recovered from his ...  


In [3]:
# 2.2 Load user demographics (movielens-1m/users.dat)
# The users.dat file is delimited by "::". We use the 'python' engine
users_df = pd.read_csv('movielens-1m/users.dat', sep='::', engine='python',
                         header=None, names=['userId', 'Gender', 'Age', 'Occupation', 'Zip-code'])
print("Users dataset shape:", users_df.shape)
print(users_df.head())

Users dataset shape: (6040, 5)
   userId Gender  Age  Occupation Zip-code
0       1      F    1          10    48067
1       2      M   56          16    70072
2       3      M   25          15    55117
3       4      M   45           7    02460
4       5      M   25          20    55455


In [4]:
# 2.3 Load ratings data (movielens-1m/ratings.dat)
# The users.dat file is delimited by "::". We use the 'python' engine
ratings_df = pd.read_csv('movielens-1m/ratings.dat', sep='::', engine='python',
                         header=None, names=['userId', 'movieId', 'rating', 'timestamp'])
print("Ratings dataset shape:", ratings_df.shape)
print(ratings_df.head())

Ratings dataset shape: (1000209, 4)
   userId  movieId  rating  timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291


# Step 3: Split Ratings into Training and Test Sets

In [5]:
# We'll use 80% of the ratings for training and 20% for testing.
train_data, test_data = train_test_split(ratings_df, test_size=0.2, random_state=42)

# Build a full user-item matrix from the entire ratings dataset.
# Rows: users, Columns: movieId. Missing ratings are filled with zeros.
R_full = ratings_df.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# For training, we start with the full matrix and then "mask" the test ratings.
R_train = R_full.copy()
# For each test record, set the corresponding entry in R_train to 0.
for idx, row in test_data.iterrows():
    R_train.loc[row['userId'], row['movieId']] = 0

# Convert training matrix to a NumPy array.
R_train = R_train.values

# Step 4: Normalize the Training Ratings Matrix and Perform SVD

In [6]:
# Compute the mean rating for each user from the training matrix.
user_ratings_mean = np.mean(R_train, axis=1)
# Demean the training matrix.
R_train_demeaned = R_train - user_ratings_mean.reshape(-1, 1)

# Perform SVD on the demeaned training matrix.
# Choose the number of latent factors (k). Adjust as needed.
k = 50
U, sigma, Vt = svds(R_train_demeaned, k=k)
sigma = np.diag(sigma)

# Reconstruct the approximated ratings matrix.
R_train_predicted = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

# Step 5: Evaluate the Model on the Test Set

In [7]:
# Create the full user-item rating matrix from ratings_df:
R_df = ratings_df.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# We'll compute RMSE for all test entries.
test_preds = []
test_truth = []

# R_df is the pivoted DataFrame we created, with columns corresponding to movieIds.
# We'll use R_df.columns.get_loc(movie) to get the column index.
for idx, row in test_data.iterrows():
    user = int(row['userId'])
    movie = int(row['movieId'])
    true_rating = row['rating']
    # Get the column index corresponding to the movieId.
    try:
        movie_idx = R_df.columns.get_loc(movie)
    except KeyError:
        # If a movie in the test set is not in the training pivot, skip it.
        continue
    pred_rating = R_train_predicted[user - 1, movie_idx]  # user indexing: userId 1 corresponds to index 0
    test_preds.append(pred_rating)
    test_truth.append(true_rating)

test_rmse = sqrt(mean_squared_error(test_truth, test_preds))
print(f"RMSE on test set: {test_rmse:.4f}\n")

RMSE on test set: 2.7405



# Step 6: Recommend Movies Using the Movie Metadata

## Recommend movies for a given user based on predicted ratings.
    
Parameters:
- user_id: The ID of the user.
- R_predicted_df: DataFrame of predicted ratings (rows: userId, columns: movieId).
- movies_df: DataFrame containing movie metadata (with descriptions).
- ratings_df: Original ratings DataFrame to determine movies already rated.
- num_recommendations: Number of recommendations to output.
    
Returns:
- A DataFrame with the top recommended movies and their details.

In [8]:
def recommend_movies(user_id, R_predicted_df, movies_df, ratings_df, num_recommendations=5):
    # Get predicted ratings for the specified user and sort them.
    user_predictions = R_predicted_df.loc[user_id].sort_values(ascending=False)
    
    # Identify movies already rated by the user.
    user_rated_movies = ratings_df[ratings_df.userId == user_id]['movieId'].tolist()
    
    # Filter out movies the user has already rated.
    recommendations = movies_df[~movies_df['movieId'].isin(user_rated_movies)].copy()
    
    # Map the predicted rating to each movie.
    recommendations['PredictedRating'] = recommendations['movieId'].map(user_predictions)
    
    # Sort recommendations by predicted rating.
    recommendations = recommendations.sort_values('PredictedRating', ascending=False)
    
    return recommendations.head(num_recommendations)

# Create a DataFrame from R_train_predicted with the same row (user) and column (movieId) labels as R_full.
R_predicted_df = pd.DataFrame(R_train_predicted, index=R_full.index, columns=R_full.columns)

# Get top 5 recommendations for a sample user (e.g., user 1).
recommended_movies = recommend_movies(1, R_predicted_df, movies_df, ratings_df, num_recommendations=5)

print("Top 5 movie recommendations for user 1:")
print(recommended_movies[['movieId', 'title', 'year', 'genres', 'description', 'PredictedRating']])

Top 5 movie recommendations for user 1:
      movieId               title  year  \
360       364       The Lion King  1994   
3682     3751         Chicken Run  2000   
2012     2081  The Little Mermaid  1989   
33         34                Babe  1995   
1262     1282            Fantasia  1940   

                                           genres  \
360                  Animation|Children's|Musical   
3682                  Animation|Children's|Comedy   
2012  Animation|Children's|Comedy|Musical|Romance   
33                        Children's|Comedy|Drama   
1262                 Animation|Children's|Musical   

                                            description  PredictedRating  
360   A young lion cub named Simba can't wait to be ...         2.449945  
3682  Having been hopelessly repressed and facing ev...         1.601483  
2012  This colorful adventure tells the story of an ...         1.585757  
33    Babe is a little pig who doesn't quite know hi...         1.532167  
1262  W